In [1]:
import chemistry as chem
import autohf as hf
from pennylane import qchem
import numpy as np
import pennylane as qml
import autograd
import autograd.numpy as anp
from autograd.differential_operators import make_jvp_reversemode as mjr
from autograd.differential_operators import make_jvp
from autograd.core import make_vjp
from tqdm.notebook import tqdm
np.set_printoptions(linewidth=300)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


In [2]:
# -15.5949085385
molecule = chem.BeH2()
R = chem.data.angs_bohr * anp.array([0.0000000000, 0.0000000000, 0.0000000000, 0.0000000000, -1.2890581334, 0.0000000000, 0.0000000000, 1.2890581334, 0.0000000000]) # Optimal geometry
structure = molecule.structure

H = molecule.hamiltonian(active_electrons=4, active_orbitals=4)

charges = [4, 1, 1]
num_elecs = 6

R1, R2, R3 = anp.array(R[0:3]), anp.array(R[3:6]), anp.array(R[6:9]) # Atomic coordiantes

In [3]:
basis_set = []
A1, A2, A3 = hf.basis_set_params("sto-3g", structure) # Gets default basis set parameters

for func in A1 + A2 + A3:
    L, exp, coeff = func
    basis_set.append(hf.AtomicBasisFunction(L, C=anp.array(coeff), A=anp.array(exp)))

In [18]:
num_act_orb = 4
num_act_elec = 4

core, active = qchem.active_space(6, 7, active_electrons=num_act_elec, active_orbitals=num_act_orb) # Prepares active space
# Function to get electronic integrals
elec_func_flat = lambda r : hf.hartree_fock(num_elecs, charges, basis_set)([r[0:3], r[3:6], r[6:9]], *([[r[0:3]]] * 5), [r[3:6]], [r[6:9]])[0]

In [19]:
#fn = lambda r : hf.electron_repulsion_tensor(basis_set)([r[0:3], r[3:6], r[6:9]], *([[r[0:3]]] * 5), [r[3:6]], [r[6:9]])

In [23]:
func = make_jvp(elec_func_flat)(-R)
vec = np.array([1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [24]:
delta = 0.001
(1 / delta) * (elec_func_flat(R + (delta/2) * vec) - elec_func_flat(R - (delta/2) * vec))

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -1.72194068e-01,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  7.95846410e-01,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  5.68367819e-06,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.39384020e-13,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-3.09239437e-04, -9.94755865e-02, -1.14452556e-13,  0.00000000e+00,  5.68367798e-06, -8.81312044e-01,  8.74880132e-13],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -3.19507253e-01,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -3.19507253e-01,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [25]:
func(vec)[1]

array([[ 1.66122576e-15, -2.99015222e-16,  2.90130033e-16,  2.81149773e-01, -2.35106114e-01, -2.39838619e-16, -4.65607909e-17],
       [ 1.20216185e-15,  1.20000278e-15, -5.73854107e-16, -8.91299797e-01,  5.24700685e-01,  5.94950591e-16, -2.50194839e-16],
       [ 2.51278308e-03, -1.71976736e-01, -6.40629098e-16,  2.30436836e-15,  2.64551230e-16,  3.59390962e-01,  1.13049239e-16],
       [-2.01745670e-16, -3.72100278e-16, -1.04728730e-15, -2.27566837e-16,  8.08703905e-17, -3.74754420e-17,  3.48980570e-16],
       [-6.76042233e-01, -4.88534487e-01, -2.19973823e-15,  3.45302975e-16, -1.13433780e-16,  4.94353455e-01, -1.10434883e-16],
       [-1.90108029e-15, -1.51134832e-15,  3.71404024e-16,  4.28522523e-01, -2.60620738e-02, -6.32648379e-17,  5.56834704e-16],
       [-1.75872520e-15, -4.66613748e-16, -1.12506062e-15,  4.28522523e-01, -2.60620738e-02,  1.14173441e-16,  3.19810959e-18]])

In [32]:
norm = lambda x : anp.sqrt(anp.sum(anp.dot(x, x)))

In [39]:
make_jvp(norm)(R)(np.array([0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]))

(3.444984394496039, -0.7071067811865476)

In [40]:
print(R)

[ 0.          0.          0.          0.         -2.43597183  0.          0.          2.43597183  0.        ]


In [ ]:
H = build_h_from_integrals(geometry, one_electron, two_electron, nuc_energy, wires, basis="sto-3g", multiplicity=1, charge=0)